# Widget Experiments / Tutorial

This notebook is primarily being used to experiment with how to use the `pymbe` widgets.

It should be fairly "clean" until the `Scratch Pad` section.

In the future, this notebook will be cleaned up and renamed "Tutorial".

## 1. Import `pymbe` and widget libraries

In [ ]:
import ipywidgets as ipyw
import traitlets as trt

import pymbe.api as pm

## 2. Create a Project Loader and Tree Explorer
> We'll also link their `elements_by_id` traitlets so as download new data, the project widget will be updated

In [ ]:
client = pm.SysML2ClientWidget(host_url="http://sysml2-sst.intercax.com")
project = pm.ProjectExplorer()
trt.link((client, "elements_by_id"), (project, "elements_by_id"))
loader = ipyw.VBox([client, project])

## 3. Create a Labelled Property Graph
> Will also link the LPG's `elements_by_id` traitlet

In [ ]:
lpg = pm.SysML2LPGWidget()
trt.link((client, "elements_by_id"), (lpg, "elements_by_id"))

## 4. ... and combine them into a single widget
> In the future, a combined widget will be offered as part of `pymbe`

In [ ]:
widget = ipyw.Tab(
    children=[loader, lpg],
    _titles={0: "Project Data", 1: "Diagramming"},
)
widget

## 5. Use the widget
...or automatically load the `Kerbal` model using the cell below

In [ ]:
client.project_selector.value = client.project_selector.options["Kerbal"]
client._download_elements()

... for example, you can select some edges and filter down the diagram to those

In [ ]:
EDGE_TYPES_TO_SELECT =  ("Superclassing", "FeatureTyping", "FeatureMembership")
lpg.edge_type_selector.value = [
    edges
    for key, edges in lpg.edge_type_selector.options.items()
    if any(
        key.startswith(edge_type)
        for edge_type in EDGE_TYPES_TO_SELECT
    )
]
lpg._update_diagram_graph()

# Scratch Pad
> **WARNING**: Anything below this point is just for experimentation purposes

In [ ]:
# Create a widget to see the qualified name of the elements nodes selected
import ipywidgets as ipyw

out = ipyw.Output(layout={'border': '1px solid black'})

def update_node_selections(*_):
    out.outputs = []
    with out:
        [
            print(element["qualifiedName"])
            for element in lpg.selected_nodes_by_type
        ]
            
lpg.node_type_selector.observe(update_node_selections, "value")

out

In [ ]:
def process_edge(source, target, edge_type, data=None):
    if edge_type in ("FeatureTyping", "FeatureMembership"):
        source, target = target, source
    return [source, target, data]

graph = nx.DiGraph()
graph.add_edges_from([
    process_edge(source, target, edge_type, data)
    for (source, target, edge_type), data in dict(client.lpg.graph.edges).items()
    if edge_type in ("Superclassing", "FeatureTyping", "FeatureMembership")
])
graph

In [ ]:
diagram.graph = graph

In [ ]:
diagram = client.lpg.make_diagram(
    graph=client.lpg.subgraph(edge_types=("Superclassing", "FeatureTyping^-1", "FeatureMembership^-1"))
)
elk_app, *_ = diagram.children
diagram

In [ ]:
# Select the root node in the diagram...
# ... or you can manually select one yourself
elk_app.selected = "5260380b-6fda-43cc-993f-5df58868edbb",

In [ ]:
first_element_selected, *_ = elk_app.selected
client.elements_by_id[first_element_selected]

# Parse JSON-LD into RDF

In [ ]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

result = client.rdf.graph
# result = g.parse(url, format='turtle')

G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
pos = nx.spring_layout(G, scale=2)
edge_labels = nx.get_edge_attributes(G, 'r')

In [ ]:
ax = plt.figure(figsize=(50,30)).gca();
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
nx.draw(G, with_labels=True, ax=ax)

# TODOs
1. Finish fleshing out the process in the `Kerbal Model.ipynb`
2. Modify the subgraph generator so it can take the value from the `Type Selector` directly
3. Improve the ipyelk diagram widget (may need to make improvements to `ipyelk`)
   * Add arrows
   * Add compartments
   * Fix layout
   * Add widget to see node details
4. Finalize the RDF formulation